In [1]:
import requests
import time
from pprint import pprint

# Demo Paas server
# ip="45.113.232.43" 

ip="localhost"
req_body = {
    "domain":"(define (domain blocksworld) (:requirements :strips) (:predicates (ON ?x ?y) (ontable ?x) (CLEAR ?x) (arm-free) (holding ?x) ) (:action pick-up :parameters (?x) :precondition (and (clear ?x) (ontable ?x) (arm-free)) :effect (and (not (ontable ?x)) (not (clear ?x)) (not (arm-free)) (holding ?x))) (:action put-down :parameters (?x) :precondition (holding ?x) :effect (and (not (holding ?x)) (clear ?x) (arm-free) (ontable ?x))) (:action stack :parameters (?x ?y) :precondition (and (holding ?x) (clear ?y)) :effect (and (not (holding ?x)) (not (clear ?y)) (clear ?x) (arm-free) (on ?x ?y))) (:action unstack :parameters (?x ?y) :precondition (and (on ?x ?y) (clear ?x) (arm-free)) :effect (and (holding ?x) (clear ?y) (not (clear ?x)) (not (arm-free)) (not (on ?x ?y)))))",
    "problem":"(define (problem blocksworld-4-2) (:domain blocksworld) (:objects b d c a ) (:init (clear a) (clear c) (clear d) (ontable a) (ontable b) (ontable d) (on c b) (arm-free)) (:goal (and (on a b) (on b c) (on c d))))"
}

In [ ]:
headers={"persistent":"true"}

In [19]:
# An example of Package API to send task and retrive plans.

def test_planner(package_name,req_body,service="solve"):
    service_url="http://"+ip+":5001/package/"+package_name+"/"+service
    solve_request = requests.post(service_url, json=req_body,headers=headers).json()
    celery_result = requests.get("http://"+ip+":5001" + solve_request['result'])
    print('Computing...')
    while celery_result.json().get("status","") == 'PENDING':
        time.sleep(2)
        celery_result = requests.get("http://"+ip+":5001" + solve_request['result'])

    pprint(celery_result.json())
    
test_planner("lama-first",req_body)

Computing...
{'result': {'call': 'lama-first domain problem',
            'output': {'sas_plan': '(unstack c b)\n'
                                   '(stack c d)\n'
                                   '(pick-up b)\n'
                                   '(stack b c)\n'
                                   '(pick-up a)\n'
                                   '(stack a b)\n'
                                   '; cost = 6 (unit cost)\n'},
            'output_type': 'generic',
            'stderr': 'bash: cannot set terminal process group (1): '
                      'Inappropriate ioctl for device\n'
                      'bash: no job control in this shell\n'
                      'bash: cannot set terminal process group (1): '
                      'Inappropriate ioctl for device\n'
                      'bash: no job control in this shell\n',
            'stdout': 'INFO     Running translator.\n'
                      'INFO     translator stdin: None\n'
                      'INFO     transl

In [20]:
# An example of Package API to send task, retrive plans with adaptor.

def test_planner_with_adaptor(package_name,req_body,service="solve"):
    service_url="http://"+ip+":5001/package/"+package_name+"/"+service
    solve_request = requests.post(service_url, json=req_body).json()
    celery_result = requests.post("http://"+ip+":5001" + solve_request['result'],json={"adaptor":"planning_editor_adaptor"})
    print('Computing...')
    while celery_result.json().get("status","") == 'PENDING':
        time.sleep(2)
        celery_result = requests.post("http://"+ip+":5001" + solve_request['result'],json={"adaptor":"planning_editor_adaptor"})
    pprint(celery_result.json())
    
test_planner_with_adaptor("lama-first",req_body)

Computing...
{'plans': [{'result': {'length': 6,
                       'logPath': 'None',
                       'output': 'INFO     Running translator.\n'
                                 'INFO     translator stdin: None\n'
                                 'INFO     translator time limit: None\n'
                                 'INFO     translator memory limit: None\n'
                                 'INFO     translator command line string: '
                                 '/usr/bin/python3 '
                                 '/workspace/downward/builds/release/bin/translate/translate.py '
                                 'domain problem --sas-file output.sas\n'
                                 'Parsing...\n'
                                 'Parsing: [0.000s CPU, 0.002s wall-clock]\n'
                                 'Normalizing task... [0.000s CPU, 0.000s '
                                 'wall-clock]\n'
                                 'Instantiating...\n'
                 

In [ ]:
# Function to test the limiter functionality

def test_limiter(package_name,req_body,service="solve"):
    service_url="http://"+ip+":5001/package/"+package_name+"/"+service
    solve_request = requests.post(service_url, json=req_body)
    return solve_request
    

import concurrent.futures
with concurrent.futures.ThreadPoolExecutor() as executor:
    futures = []
    for i in range(30):
        futures.append(executor.submit(test_limiter, "lama",req_body))
    for future in concurrent.futures.as_completed(futures):
        print(future.result())